In [3]:
import pickle

with open('C:\\Users\\geets\\Desktop\\code\\final_ml\\image_captioning_system\\features.pkl', 'rb') as f:
    features = pickle.load(f)



In [29]:
def load_doc(filename):
    with open(filename, 'r') as file:
        text = file.read()
    return text


In [30]:
cleaned_doc = load_doc("descriptions.txt")


In [31]:
def load_descriptions(doc):
    descriptions = {}
    for line in doc.split('\n'):
        tokens = line.split()
        if len(tokens) < 2:
            continue
        image_id, image_desc = tokens[0], tokens[1:]
        image_id = image_id.split('.')[0]
        desc = ' '.join(image_desc)
        if image_id not in descriptions:
            descriptions[image_id] = []
        descriptions[image_id].append(desc)
    return descriptions

descriptions = load_descriptions(cleaned_doc)



In [27]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add


In [32]:
tokenizer = create_tokenizer(descriptions)
vocab_size = len(tokenizer.word_index) + 1



In [15]:
max_length = max(len(d.split()) for desc_list in descriptions.values() for d in desc_list)


In [40]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def data_generator(descriptions, features, tokenizer, max_length, vocab_size):
    while True:
        for key, desc_list in descriptions.items():
            # Ensure feature is a NumPy array
            feature = np.array(features[key])
            if feature.ndim == 2:
                feature = feature[0]  # Remove extra dimension if present

            for desc in desc_list:
                seq = tokenizer.texts_to_sequences([desc])[0]
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]

                    # Pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]

                    # One-hot encode output word
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                    # Return arrays, not lists
                    yield ([np.array(feature), np.array(in_seq)], np.array(out_seq))



In [18]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

def define_model(vocab_size, max_length):
    # Feature extractor (ResNet feature vector input)
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    # Sequence processor (Caption input)
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    # Decoder (combine feature + caption input)
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

model = define_model(vocab_size, max_length)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 34)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 2048)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 34, 256)   │  2,244,352 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 2048)      │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 34, 256)   │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 34)        │          0 │ input_layer_1[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    524,544 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 256)       │    525,312 │ dropout_1[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 256)       │          0 │ dense[0][0],      │
│                     │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     65,792 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 8767)      │  2,253,119 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,613,119 (21.41 MB)

 Trainable params: 5,613,119 (21.41 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
def data_generator(descriptions, features, tokenizer, max_length, vocab_size):
    while True:
        for key, desc_list in descriptions.items():
            feature = features[key]
            for desc in desc_list:
                seq = tokenizer.texts_to_sequences([desc])[0]
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    yield ([feature, in_seq], out_seq)  # <- this must be a tuple



In [44]:
generator = data_generator(descriptions, features, tokenizer, max_length, vocab_size)
model.fit(generator, epochs=20, steps_per_epoch=300, verbose=1)




TypeError: `output_signature` must contain objects that are subclass of `tf.TypeSpec` but found <class 'list'> which is not.

In [47]:
dataset = tf.data.Dataset.from_generator(
    wrapped_generator,
    output_signature=output_signature
)

# Optional: Add batching and prefetching for performance
dataset = dataset.batch(1).prefetch(tf.data.AUTOTUNE)


In [54]:
def wrapped_generator():
    for key, desc_list in descriptions.items():
        feature = features[key]
        for desc in desc_list:
            seq = tokenizer.texts_to_sequences([desc])[0]
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                # Convert to float32 arrays explicitly
                yield (
                    {
                        "input_layer": np.array(feature, dtype=np.float32),
                        "input_layer_1": np.array(in_seq, dtype=np.int32)
                    },
                    np.array(out_seq, dtype=np.float32)
                )



In [55]:
dataset = tf.data.Dataset.from_generator(
    wrapped_generator,
    output_signature=output_signature
).batch(1).prefetch(tf.data.AUTOTUNE)


In [57]:
model.fit(dataset, epochs=20, steps_per_epoch=300)


Epoch 1/20


ValueError: Exception encountered when calling Functional.call().

[1mThe structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor', 'keras_tensor_3']
Received: inputs={'input_layer': 'Tensor(shape=(None, 2048))', 'input_layer_1': 'Tensor(shape=(None, 34))'}[0m

Arguments received by Functional.call():
  • inputs={'input_layer': 'tf.Tensor(shape=(None, 2048), dtype=float32)', 'input_layer_1': 'tf.Tensor(shape=(None, 34), dtype=int32)'}
  • training=True
  • mask={'input_layer': 'None', 'input_layer_1': 'None'}

In [58]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, add
from tensorflow.keras.models import Model

# Make sure names match what you use in the generator!
image_input = Input(shape=(2048,), name="input_layer")
text_input = Input(shape=(max_length,), name="input_layer_1")

# Example model pipeline (simplified for clarity)
img_dense = Dense(256, activation='relu')(image_input)
txt_emb = Embedding(vocab_size, 256, mask_zero=True)(text_input)
txt_lstm = LSTM(256)(txt_emb)

decoder = add([img_dense, txt_lstm])
decoder = Dense(256, activation='relu')(decoder)
output = Dense(vocab_size, activation='softmax')(decoder)

model = Model(inputs=[image_input, text_input], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [65]:
def wrapped_generator():
    for key, desc_list in descriptions.items():
        feature = features[key].flatten()[:2048]  # Flatten then slice to (2048,)
        for desc in desc_list:
            seq = tokenizer.texts_to_sequences([desc])[0]
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                yield (
                    {
                        "input_layer": np.array(feature, dtype=np.float32),         # (2048,)
                        "input_layer_1": np.array(in_seq, dtype=np.int32),         # (max_length,)
                    },
                    np.array(out_seq, dtype=np.float32)                            # (vocab_size,)
                )



In [67]:
def wrapped_generator():
    for key, desc_list in descriptions.items():
        feature = features[key].flatten()[:2048]  # Ensure shape is (2048,)
        for desc in desc_list:
            seq = tokenizer.texts_to_sequences([desc])[0]
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                # Debug print statements (just once)
                print("feature shape:", feature.shape)
                print("in_seq shape:", in_seq.shape)
                print("out_seq shape:", out_seq.shape)
                break  # break after first example for inspection
            break
        break


In [ ]:
def wrapped_generator():
    for key, desc_list in descriptions.items():
        feature = features[key].flatten()[:2048]
        for desc in desc_list:
            seq = tokenizer.texts_to_sequences([desc])[0]
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                # Debug prints
                print("feature shape:", feature.shape)
                print("in_seq shape:", in_seq.shape)
                print("out_seq shape:", out_seq.shape)

                yield (
                    {
                        "input_layer": feature,
                        "input_layer_1": in_seq
                    },
                    out_seq
                )
                 # Stop after first sample to test


In [70]:
gen = wrapped_generator()
next(gen)


feature shape: (2048,)
in_seq shape: (34,)
out_seq shape: (8764,)


({'input_layer': array([1.9046047 , 0.        , 0.        , ..., 0.49275672, 0.        ,
         0.        ], dtype=float32),
  'input_layer_1': array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 39])},
 array([0., 1., 0., ..., 0., 0., 0.]))

In [72]:
dataset = tf.data.Dataset.from_generator(
    wrapped_generator,
    output_signature=(
        {
            "input_layer": tf.TensorSpec(shape=(2048,), dtype=tf.float32),
            "input_layer_1": tf.TensorSpec(shape=(max_length,), dtype=tf.int32),
        },
        tf.TensorSpec(shape=(vocab_size,), dtype=tf.float32)
    )
).repeat().batch(1).prefetch(tf.data.AUTOTUNE)


model.fit(dataset, epochs=20, steps_per_epoch=300)



Epoch 1/20
feature shape: (2048,)
in_seq shape: (34,)
out_seq shape: (8764,)
feature shape: (2048,)
in_seq shape: (34,)
out_seq shape: (8764,)
  1/300 ━━━━━━━━━━━━━━━━━━━━ 49s 167ms/step - loss: 0.0000e+00feature shape: (2048,)
in_seq shape: (34,)
out_seq shape: (8764,)
  2/300 ━━━━━━━━━━━━━━━━━━━━ 21s 72ms/step - loss: 0.0000e+00 feature shape: (2048,)
in_seq shape: (34,)
out_seq shape: (8764,)
  3/300 ━━━━━━━━━━━━━━━━━━━━ 20s 70ms/step - loss: 0.0000e+00feature shape: (2048,)
in_seq shape: (34,)
out_seq shape: (8764,)
  4/300 ━━━━━━━━━━━━━━━━━━━━ 22s 78ms/step - loss: 0.0000e+00feature shape: (2048,)
in_seq shape: (34,)
out_seq shape: (8764,)
  5/300 ━━━━━━━━━━━━━━━━━━━━ 21s 75ms/step - loss: 0.0000e+00feature shape: (2048,)
in_seq shape: (34,)
out_seq shape: (8764,)
  6/300 ━━━━━━━━━━━━━━━━━━━━ 22s 78ms/step - loss: 0.0000e+00feature shape: (2048,)
in_seq shape: (34,)
out_seq shape: (8764,)
  7/300 ━━━━━━━━━━━━━━━━━━━━ 22s 77ms/step - loss: 0.0000e+00feature shape: (2048,)
in_seq sh

In [80]:
from tensorflow.keras.preprocessing.text import Tokenizer


# Load captions from Flickr8k.token.txt
def load_captions(file_path):
    captions = {}
    with open(file_path, 'r') as f:
        for line in f:
            tokens = line.strip().split('\t')
            if len(tokens) != 2:
                continue
            image_id, caption = tokens
            image_id = image_id.split('#')[0]
            caption = f'startseq {caption.lower()} endseq'
            captions.setdefault(image_id, []).append(caption)
    return captions

captions_dict = load_captions('Flickr8k.token.txt')

# Flatten all captions
all_captions = []
for cap_list in captions_dict.values():
    all_captions.extend(cap_list)

# Fit tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)

# Save tokenizer for later use
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


In [81]:
import pickle
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model


In [82]:
# Load tokenizer
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# Load features (VGG16 features)
with open("features.pkl", "rb") as f:
    features = pickle.load(f)


In [99]:
# Map index to word
index_to_word = {v: k for k, v in tokenizer.word_index.items()}
vocab_size = len(tokenizer.word_index) + 1
max_length = 34  # Use the same as during training

def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def generate_caption(model, tokenizer, photo_feature, max_length):
    in_text = 'startseq'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo_feature, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = word_for_id(yhat, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text


In [89]:
!pip install tqdm


  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [90]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
import os
import pickle
from tqdm import tqdm

# Load model
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
model.trainable = False

# Path to image directory
images_path = r"C:\Users\geets\Desktop\code\new_ml\Flickr8k_Dataset\Flicker8k_Dataset"

# Output dict
features = {}

# List image files
image_files = [f for f in os.listdir(images_path) if f.lower().endswith('.jpg')]

# Process each image
for idx, img_name in enumerate(tqdm(image_files, desc="Extracting features")):
    img_path = os.path.join(images_path, img_name)
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Extract feature
    feature = model.predict(img_array, verbose=0)
    image_id = img_name.split('.')[0]
    features[image_id] = feature.squeeze()

    # Print every 100 images
    if (idx + 1) % 100 == 0:
        print(f"[INFO] Processed {idx + 1} images")

# Save features
with open("resnet_features.pkl", "wb") as f:
    pickle.dump(features, f)

print("✅ Feature extraction complete!")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


Extracting features:   1%|          | 100/8091 [00:21<49:01,  2.72it/s]

[INFO] Processed 100 images


Extracting features:   2%|▏         | 201/8091 [00:50<17:42,  7.43it/s]

[INFO] Processed 200 images


...

[INFO] Processed 7900 images


Extracting features:  99%|█████████▉| 8001/8091 [24:05<00:15,  5.89it/s]

[INFO] Processed 8000 images


Extracting features: 100%|██████████| 8091/8091 [24:21<00:00,  5.54it/s]


✅ Feature extraction complete!


In [91]:
import pickle

# Load tokenizer (no change)
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# Load the NEW ResNet-based features (2048-dim)
with open("resnet_features.pkl", "rb") as f:  # Make sure this matches your filename
    features = pickle.load(f)


In [92]:
photo_id = list(features.keys())[0]  # You can also try others
photo_feature = features[photo_id]   # Shape should be (1, 2048) or (2048,)


In [94]:
photo_feature = photo_feature.reshape((2048,))


In [100]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np

# Load ResNet50 model without top classification layer
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_resnet_feature(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    feature = resnet_model.predict(img_array, verbose=0)
    return feature  # shape will be (1, 2048)

# Usage:
img_path = r"C:\Users\geets\Desktop\code\final_ml\image_captioning_system\35506150_cbdb630f4f.jpg"
photo_feature = extract_resnet_feature(img_path)  # shape: (1, 2048)


In [105]:
model.save("caption_model.h5")


In [107]:
model.summary()


Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, None,      │          0 │ input_layer_4[0]… │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,472 │ conv1_pad[0][0]   │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, None,      │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, None,      │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, None,      │          0 │ conv2_block1_2_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, None,      │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, None,      │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, None,      │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [108]:
model.summary()


Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, None,      │          0 │ input_layer_4[0]… │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,472 │ conv1_pad[0][0]   │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, None,      │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, None,      │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, None,      │          0 │ conv2_block1_2_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, None,      │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, None,      │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, None,      │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [109]:
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from keras.models import Model

# Image features input
inputs1 = Input(shape=(2048,), name="input_layer")
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# Caption input
inputs2 = Input(shape=(max_length,), name="input_layer_1")
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = LSTM(256)(se1)

# Decoder (merge + output)
decoder1 = add([fe2, se2])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [110]:
model.fit(dataset, epochs=20, steps_per_epoch=300)


Epoch 1/20


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 8764), output.shape=(None, 8496)

In [111]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

# Fit tokenizer on all captions
all_captions = [desc for desc_list in descriptions.values() for desc in desc_list]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)

vocab_size = len(tokenizer.word_index) + 1  # +1 for padding


In [112]:
def wrapped_generator():
    for key, desc_list in descriptions.items():
        feature = features[key].flatten()[:2048].astype(np.float32)
        for desc in desc_list:
            seq = tokenizer.texts_to_sequences([desc])[0]
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                yield (
                    {
                        "image_input": feature,
                        "text_input": in_seq
                    },
                    out_seq
                )


In [113]:
import tensorflow as tf

dataset = tf.data.Dataset.from_generator(
    wrapped_generator,
    output_signature=(
        {
            "image_input": tf.TensorSpec(shape=(2048,), dtype=tf.float32),
            "text_input": tf.TensorSpec(shape=(max_length,), dtype=tf.int32),
        },
        tf.TensorSpec(shape=(vocab_size,), dtype=tf.float32)
    )
).repeat().batch(1).prefetch(tf.data.AUTOTUNE)


In [114]:
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, add
from tensorflow.keras.models import Model

# Image feature input
image_input = Input(shape=(2048,), name="image_input")
img_dropout = Dropout(0.5)(image_input)
img_dense = Dense(256, activation='relu')(img_dropout)

# Text sequence input
text_input = Input(shape=(max_length,), name="text_input")
txt_embed = Embedding(input_dim=vocab_size, output_dim=256, mask_zero=True)(text_input)
txt_dropout = Dropout(0.5)(txt_embed)
txt_lstm = LSTM(256)(txt_dropout)

# Decoder
decoder = add([img_dense, txt_lstm])
decoder = Dense(256, activation='relu')(decoder)
outputs = Dense(vocab_size, activation='softmax')(decoder)

model = Model(inputs=[image_input, text_input], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 34)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ image_input         │ (None, 2048)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 34, 256)   │  2,243,584 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 2048)      │          0 │ image_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 34, 256)   │          0 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_3         │ (None, 34)        │          0 │ text_input[0][0]  │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 256)       │    524,544 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 256)       │    525,312 │ dropout_4[0][0],  │
│                     │                   │            │ not_equal_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 256)       │          0 │ dense_9[0][0],    │
│                     │                   │            │ lstm_3[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 256)       │     65,792 │ add_3[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 8764)      │  2,252,348 │ dense_10[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,611,580 (21.41 MB)

 Trainable params: 5,611,580 (21.41 MB)

 Non-trainable params: 0 (0.00 B)

In [115]:
model.fit(dataset, epochs=20, steps_per_epoch=300)


Epoch 1/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 18s 55ms/step - loss: 6.3134
Epoch 2/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 17s 57ms/step - loss: 5.9473
Epoch 3/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 17s 57ms/step - loss: 5.8992
Epoch 4/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 17s 57ms/step - loss: 5.8304
Epoch 5/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - loss: 5.9805
Epoch 6/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 17s 57ms/step - loss: 6.0697
Epoch 7/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - loss: 6.2373
Epoch 8/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 19s 63ms/step - loss: 5.7493
Epoch 9/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - loss: 5.7519
Epoch 10/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 17s 58ms/step - loss: 6.7765
Epoch 11/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 17s 58ms/step - loss: 5.7373
Epoch 12/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 17s 58ms/step - loss: 6.4185
Epoch 13/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 17s 58ms/step - loss: 6.1683
Epoch 14/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 17s 58ms/step - loss: 6.5870
Epoch 15/20
300/300 ━━━━━━━━━

In [116]:
model.save("caption_model_final.h5")


In [117]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np

def generate_caption(model, image_feature, tokenizer, max_length):
    in_text = 'startseq'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([np.array([image_feature]), sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = tokenizer.index_word.get(yhat)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text.replace('startseq', '').replace('endseq', '').strip()


In [118]:
# Load the extracted ResNet features
with open("resnet_features.pkl", "rb") as f:
    features = pickle.load(f)

# Generate a caption for the first image feature
img_key = list(features.keys())[0]
caption = generate_caption(model, features[img_key], tokenizer, max_length)
print("Caption:", caption)



Caption: on in on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on


In [129]:
import pickle

# Load extracted features
with open("resnet_features.pkl", "rb") as f:
    features = pickle.load(f)

# List all available image keys
print("Available images:", list(features.keys())[:45])  # show 5 keys for now


Available images: ['1000268201_693b08cb0e', '1001773457_577c3a7d70', '1002674143_1b742ab4b8', '1003163366_44323f5815', '1007129816_e794419615', '1007320043_627395c3d8', '1009434119_febe49276a', '1012212859_01547e3f17', '1015118661_980735411b', '1015584366_dfcec3c85a', '101654506_8eb26cfb60', '101669240_b2d3e7f17b', '1016887272_03199f49c4', '1019077836_6fc9b15408', '1019604187_d087bf9a5f', '1020651753_06077ec457', '1022454332_6af2c1449a', '1022454428_b6b660a67b', '1022975728_75515238d8', '102351840_323e3de834', '1024138940_f1fefbdce1', '102455176_5f8ead62d5', '1026685415_0431cbf574', '1028205764_7e8df9a2ea', '1030985833_b0902ea560', '103106960_e8a41d64f8', '103195344_5d2dc613a3', '103205630_682ca7285b', '1032122270_ea6f0beedb', '1032460886_4a598ed535', '1034276567_49bb87c51c', '104136873_5b5d41be75', '1042020065_fb3d3ba5ba', '1042590306_95dea0916c', '1045521051_108ebc19be', '1048710776_bb5b0a5c7c', '1052358063_eae6744153', '105342180_4d4a40b47f', '1053804096_ad278b25f1', '1055623002_819

In [132]:
img_key = list(features.keys())[41]  # try index 3 or another
caption = generate_caption(model, features[img_key], tokenizer, max_length)
print("Caption:", caption)


Caption: ('on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on', [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])


In [124]:
def generate_caption(model, image, tokenizer, max_length):
    in_text = '<start>'
    sequence = []
    
    for _ in range(max_length):
        sequence_ids = tokenizer.texts_to_sequences([in_text])[0]
        sequence_ids = pad_sequences([sequence_ids], maxlen=max_length)
        
        yhat = model.predict([np.expand_dims(image, 0), sequence_ids], verbose=0)
        yhat_id = np.argmax(yhat)
        sequence.append(yhat_id)

        word = tokenizer.index_word.get(yhat_id, None)
        if word is None:
            break
        in_text += ' ' + word
        if word == '<end>':
            break

    final_caption = in_text.split()
    final_caption = final_caption[1:-1] if '<end>' in final_caption else final_caption[1:]
    return ' '.join(final_caption), sequence  # Return both


In [126]:
from collections import Counter

word_counts = tokenizer.word_counts
print(Counter(word_counts).most_common(10))


[('in', 37950), ('the', 36838), ('on', 21487), ('is', 18690), ('and', 17704), ('dog', 16272), ('with', 15530), ('man', 14532), ('of', 13426), ('two', 11278)]


In [127]:
def beam_search_predict(model, image, tokenizer, max_length=34, beam_index=3):
    start = [tokenizer.word_index['<start>']]
    sequences = [[start, 0.0]]

    while len(sequences[0][0]) < max_length:
        all_candidates = []
        for seq, score in sequences:
            padded = pad_sequences([seq], maxlen=max_length)
            yhat = model.predict([np.expand_dims(image, 0), padded], verbose=0)
            top_idx = np.argsort(yhat[0])[-beam_index:]

            for idx in top_idx:
                new_seq = seq + [idx]
                new_score = score + np.log(yhat[0][idx] + 1e-9)
                all_candidates.append([new_seq, new_score])

        sequences = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)[:beam_index]

    final_seq = sequences[0][0]
    final_caption = [tokenizer.index_word.get(i, '') for i in final_seq]
    return ' '.join(final_caption[1:-1])


In [128]:
img_key = list(features.keys())[0]
caption = beam_search_predict(model, features[img_key], tokenizer)
print("Beam Search Caption:", caption)


KeyError: '<start>'